# True Label

In [34]:
import os
import pandas as pd
from PIL import Image
import numpy as np

# 이미지 파일들이 저장된 디렉토리 경로
directory = "data4/test/masks"

# 디렉토리 내에 있는 모든 이미지 파일 경로 리스트
img_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.jpg') or f.endswith('.png')]

# 이미지 파일들을 3차원 numpy array로 불러오기
images = []
for f in img_files:
    img = Image.open(f)
    img_arr = np.asarray(img)
    images.append(img_arr)

# 3차원 numpy array를 데이터프레임으로 변환
df_list = []
for i, img in enumerate(images):
    height, width, channels = img.shape
    img_df = pd.DataFrame(img.reshape(height*width, channels))
    img_df['Image'] = i+1  # 이미지 번호
    df_list.append(img_df)

df = pd.concat(df_list, axis=0)
df.columns = ['Red', 'Green', 'Blue', 'Image']
df.reset_index(inplace=True)
df.drop(['Image','index'],axis=1,inplace=True)
df.reset_index(inplace=True)
df.columns = ['Image','Red', 'Green', 'Blue']
df.to_csv('true_label.csv', index=False)


In [35]:
df

,Image,Red,Green,Blue
0,0,0,0,0
1,1,0,0,0
2,2,0,0,0
3,3,0,0,0
4,4,0,0,0
...,...,...,...,...
2376836,2376836,0,255,0
2376837,2376837,0,255,0
2376838,2376838,0,0,0
2376839,2376839,0,0,0


In [ ]:
true = pd.read_csv('true_label.csv').drop(['Image'],axis=1).drop_duplicates()

# Predcit image

In [17]:
from PIL import Image
import numpy as np


img = Image.open('data4/train/masks/1.png')
mask = np.asarray(img)
mask = mask.reshape(-1, mask.shape[-1])
unique_value = np.unique(mask, axis=0).tolist()
unique_values = [tuple(sublist) for sublist in unique_value]

In [18]:
from typing import List, Tuple

def find_closest_pixel(pixel, colors):
    """
    주어진 픽셀 값에 대해 리스트 내에서 가장 작은 차이를 가지는 색상을 찾아서 반환하는 함수
    
    :param pixel: 비교하고자 하는 픽셀 값
    :param colors: 비교하고자 하는 색상 리스트
    :return: 가장 작은 차이를 가지는 색상 값
    """
    min_diff = float('inf')  # 최소 차이 값
    closest_color = None  # 가장 작은 차이를 가지는 색상 값
    for color in colors:
        # 각 색상 값과 픽셀 값의 차이 계산
        diff = sum([abs(pixel[i] - color[i]) for i in range(3)])
        # 현재까지의 최소 차이보다 더 작은 차이가 나오면 갱신
        if diff < min_diff:
            min_diff = diff
            closest_color = color
    return closest_color

In [19]:
import cv2
import numpy as np

def replace_pixels_with_closest_color(img,colors):
    """
    이미지 내의 모든 픽셀 값을 주어진 색상 리스트에서 가장 작은 차이를 가지는 색상으로 대체하는 함수
    
    :param img: 대상 이미지
    :param colors: 대체하고자 하는 색상 리스트
    :return: 대체된 이미지
    """
    # 이미지를 1차원 벡터 형태로 변환
    pixels = img.reshape(-1, 3)
    # 각 픽셀 값에 대해 가장 작은 차이를 가지는 색상으로 대체
    new_pixels = [find_closest_pixel(pixel, colors) for pixel in pixels]
    # 변환된 픽셀 값을 이미지 형태로 재조합하여 반환
    return np.array(new_pixels, dtype=np.uint8).reshape(img.shape)

In [21]:
import os
import pandas as pd
from PIL import Image
import numpy as np
from tqdm import tqdm

# 이미지 파일들이 저장된 디렉토리 경로
directory = "Output"

# 디렉토리 내에 있는 모든 이미지 파일 경로 리스트
img_files = [os.path.join(directory, f) for f in os.listdir(directory)]

# 이미지 파일들을 3차원 numpy array로 불러오기
images = []
for f in tqdm(img_files):
    img = cv2.imread(f)
    img = replace_pixels_with_closest_color(img,unique_values)
    img_arr = np.array(img)
    images.append(img_arr)

# 3차원 numpy array를 데이터프레임으로 변환
df_list = []
for i, img in enumerate(images):
    height, width, channels = img.shape
    img_df = pd.DataFrame(img.reshape(height*width, channels))
    img_df['Image'] = i+1  # 이미지 번호
    df_list.append(img_df)

df = pd.concat(df_list, axis=0)
df.columns = ['Red', 'Green', 'Blue', 'Image']
df.reset_index(inplace=True)
df.drop(['Image','index'],axis=1,inplace=True)
df.reset_index(inplace=True)
df.columns = ['Image','Red', 'Green', 'Blue']
df.to_csv('submission_label.csv', index=False)


100%|██████████| 5/5 [02:41<00:00, 32.29s/it]


In [22]:
df

,Image,Red,Green,Blue
0,0,0,128,128
1,1,0,128,128
2,2,0,128,128
3,3,0,0,255
4,4,0,0,255
...,...,...,...,...
2376836,2376836,128,128,128
2376837,2376837,0,255,0
2376838,2376838,255,0,255
2376839,2376839,0,128,128


In [10]:
img

array([[[128, 128,   0],
        [128, 128,   0],
        [255,   0,   0],
        ...,
        [128, 128,   0],
        [128, 128,   0],
        [128, 128,   0]],

       [[128, 128,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [  0,   0,   0],
        [128, 128,   0],
        [128, 128,   0]],

       [[  0,   0, 128],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ...,

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [  0,   0,   0],
        [128, 128,   0],
        [128, 128,   0]],

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [128, 128,   0],
        [128, 128,   0],
        [128, 128,   0]],

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [  0, 128, 128]]